In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-

In [3]:
import nltk
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
no_finetuned = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
inputs = tokenizer("What dishes can we make with beef and tomato", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['mashed potatoes']


In [6]:
finetuned = T5ForConditionalGeneration.from_pretrained("gdrive/My Drive/T5", local_files_only=True)

In [7]:
outputs = finetuned.generate(**inputs, max_length=100)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['You can make Beef And Tomato Soup. Here is the recipe: Mix beef and tomato in a large bowl. Add beef and tomato mixture and mix well. Pour into a greased 9 x 13-inch pan. Bake at 350° for 30 minutes.']


In [12]:
inputs = tokenizer("What dishes can we make with shrimp, cucumber, tomato", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['shrimp, cucumber, tomato, steamed rice, steamed rice, steamed']


In [13]:
outputs = finetuned.generate(**inputs, max_length=100)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['You can make Shrimp Salad. Here is the recipe: Mix all ingredients together and refrigerate overnight.']


In [14]:
inputs = tokenizer("What dishes can we make with potato, salmon, and Broccoli", return_tensors="pt")
outputs = no_finetuned.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['salmon and broccoli']


In [17]:
outputs = finetuned.generate(**inputs, max_length=100)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['You can make Salmon Casserole. Here is the recipe: Cook salmon in a skillet over medium heat until tender. Drain and drain. Add broccoli and cook until tender. Add salmon and cook until tender. Add salmon and cook until tender. Add broccoli and cook until tender. Serves 6 to 8 servings.']
